In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import gen_css
import add_current

In [2]:
ori_data = requests.get('https://www.econometricsociety.org/publications/econometrica')

In [3]:
data = BeautifulSoup(ori_data.text, 'html.parser')

In [4]:
current = data.find(string=re.compile('Latest Issue'))
issue = current[14:]

In [10]:
add_current.add(issue, time.ctime(), 'ecma/ecma_' + issue + '.html')

In [6]:
article = data.find_all(class_='article')

In [7]:
title = []
authors = []
ab_links = []
for i in article:
    title.append(i.contents[1].text)
    authors.append(i.contents[3])
    try:
        ab_links.append(i.find(string=re.compile('Abstract')).parent.get('href'))
    except:
        ab_links.append('no_ab')

In [8]:
author = []
for i in authors:
    for child in i.children:
        if child.name is None:
            author.append(child)
        elif child.name == 'br':
            pass

In [9]:
abstract = []
for ab_link in ab_links:
    if ab_link == 'no_ab':
        abstract.append('no_ab')
    else:
        try:
            abstract_page = requests.get('https://www.econometricsociety.org' + ab_link)
            ab_data = BeautifulSoup(abstract_page.text, 'html.parser')
            abstract.append(ab_data.find(class_='abstract').text)
        except:
            abstract.append('error')

In [ ]:
articles = [*zip(title, author, abstract)]

In [ ]:
with open('ecma/ecma_' + issue + '.html','w',encoding='UTF-8') as f:
    f.write(gen_css.gen_content_from_list(title))
    f.write(gen_css.gen_abstract_list(list(articles)))